<a href="https://colab.research.google.com/github/fopamesmin/4MAc/blob/main/_Liga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as BS
import logging
from google.colab import drive

# Constants
sofifa_url = "https://www.laliga.com/en-ES/laliga-easports/standing"
COLUMNS = ["position", "team", "Points", "W", "D", "L", "GF", "GA", "GD"]
ID_seasonId = ["2020/2021", "2021/2022", "2022/2023"]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
logging.basicConfig(level=logging.INFO)

def fetch_data(season):
    teams_list = []
    response = req.get(sofifa_url, headers=headers)
    if response.status_code == req.codes.ok:
        soup = BS(response.content, "lxml")
        rows = soup.select("div.styled__StandingTabBody-sc-e89col-0.isRHqh")
        for row in rows:
            team_data = []
            try:
                position = row.find("div", class_="styled__Td-sc-e89col-10 fTFWtb").find("p").text.strip()
                team = row.find("div", class_="styled__ShieldContainer-sc-1opls7r-0 eIaTDi shield-desktop").find("p").text.strip()
                points = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[0].find("p").text.strip()
                w = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[1].find("p").text.strip()
                d = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[2].find("p").text.strip()
                l = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[3].find("p").text.strip()
                gf = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[4].find("p").text.strip()
                ga = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[5].find("p").text.strip()
                gd = row.find_all("div", class_="styled__Td-sc-e89col-10 feNufd")[6].find("p").text.strip()

                # Ajouter le dictionnaire pour renommer les colonnes
                rename_columns = {
                    'W': 'Won',
                    'D': 'Drawn',
                    'L': 'Lost'
                }

                # Étendre les données de l'équipe avec la saison et les colonnes renommées
                team_data.extend([position, team, points, w, d, l, gf, ga, gd])
                teams_list.append(team_data)
            except Exception as e:
                logging.error(f"Error parsing row: {e}")
                continue
    else:
        logging.error(f"Failed to retrieve data for season {season}")
    return teams_list

# Monter Google Drive
drive.mount('/content/drive')

# Extraction des données pour toutes les saisons
all_teams_list = []
for season in ID_seasonId:
    all_teams_list.extend(fetch_data(season))

# Convertir la liste en DataFrame pandas avec les noms de colonnes renommés
df = pd.DataFrame(all_teams_list, columns=["position", "team", "Points", "Won", "Drawn", "Lost", "GF", "GA", "GD"])

# Enregistrer le dataframe dans Google Drive avec les nouveaux noms de colonnes
file_path = '/content/drive/My Drive/laliga_data.csv'
df.to_csv(file_path, header=True, index=False)

print(f"Données enregistrées dans Google Drive à l'emplacement : {file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Données enregistrées dans Google Drive à l'emplacement : /content/drive/My Drive/laliga_data.csv


In [8]:
import pandas as pd
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Chemin vers votre fichier CSV existant dans Google Drive
file_path = '/content/drive/My Drive/laliga_data.csv'

# Chargement du DataFrame depuis le fichier CSV existant
df = pd.read_csv(file_path)

# Ajout de la colonne Played en calculant la somme de Won, Drawn et Lost
df['Played'] = df['W'] + df['D'] + df['L']

# Dictionnaire pour renommer les colonnes
rename_columns = {
    'W': 'Won',
    'D': 'Drawn',
    'L': 'Lost'
}

# Renommer les colonnes
df.rename(columns=rename_columns, inplace=True)

# Sauvegarder le DataFrame avec les colonnes renommées et la colonne Played dans un nouveau fichier CSV
new_file_path = '/content/drive/My Drive/laliga_data.csv'
df.to_csv(new_file_path, index=False)

print(f"Données avec colonnes renommées et colonne Played ajoutée enregistrées dans Google Drive : {new_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Données avec colonnes renommées et colonne Played ajoutée enregistrées dans Google Drive : /content/drive/My Drive/laliga_data.csv
